In [1]:
import pandas as pd
import networkx as nx
df = pd.read_excel(r'C:\Users\RIC_ZX\Desktop\papers\ZLW\(申请号去重后）美国和欧盟 已筛选TAC_(poly butylene adipate-co-terephthalate) OR TAC_(PBAT).xlsx')
df['IPC主分类号'] = df['IPC主分类号'].str[:4]
df = df.drop([0, 1])
df

,序号,公开(公告)号,标题,法律状态/事件,申请日,发明人,申请号,权利要求,简单同族,简单同族成员数量,...,优先权国家/地区,被引用专利,被引用专利数量,引用专利,引用专利数量,非专利引文数量,许可类型,5年内被引用次数,诉讼案件数,质押人
2,1057,US675631A,Thio-urea compound and process of making same.,期限届满,1900-12-31,ADOLF ISRAEL | RICHARD KOTHE,US1900041628,Having now described our invention and in 20 w...,US675631A,1,...,-,US5144064A,NaN,-,0,0,-,0,0,-
3,1037,US20230250275A1,Algae thermoplastic composition and process of...,公开,2023-03-07,"ZELLER, MARK ASHTON | HUNT, RYAN",US18/180014,1. An algae-based thermoplastic composition co...,US20230250275A1,1,...,US,-,NaN,-,0,0,-,0,0,-
4,1018,US20230212537A1,Novel esterases and uses thereof,公开,2022-12-22,"DAVID, BENOÎT | ANDRE, ISABELLE | BEN KHALED, ...",US18/145073,1. An esterase which (i) has at least 95% iden...,CA3107517A1 | CN112654701A | EP3830254A1 | IN2...,10,...,EP,-,NaN,-,0,0,-,0,0,-
5,1001,US20230127747A1,Biodegradable and Industrially Compostable Inj...,实质审查 | 权利转移,2022-12-23,"FALKEN, ROBERT",US18/087934,1-20. (canceled)_x000D_\n21. A method for manu...,AU2019272664A1 | BR112020023590A2 | CN11251277...,12,...,US,-,NaN,US20100198133A1 | US20130154143A1,2,0,-,0,0,-
6,1017,US20230208143A1,System and method for smoothing photovoltaic g...,公开,2022-12-22,"AREDES, MAURICIO | BARBOSA ROLIM, LUÍS GUILHER...",US18/087199,1. A system for smoothing photovoltaic generat...,BR102021026269A2 | US20230208143A1,2,...,BR,-,NaN,-,0,0,-,0,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,21,EP2294125A2,Starch-containing thermoplastic or elastomer c...,撤回-视为撤回,2009-06-12,"MENTINK, LÉON | GIMENEZ, JÉRÔME | LAGNEAUX, DI...",EP2009761936,REVENDICATIONS_x000D_\n1. Composition thermopl...,AU2009259118A1 | BRPI0914417A2 | CA2726860A1 |...,9,...,FR,-,NaN,-,0,0,-,0,0,-
446,20,EP2247661A2,Method for preparing thermoplastic composition...,撤回-视为撤回,2009-01-29,"MENTINK, LÉON | LAGNEAUX, DIDIER | GIMENEZ, JÉ...",EP2009705988,REVENDICATIONS_x000D_\n1. Composition à base d...,AU2009208830A1 | AU2009208830B2 | BRPI0907038A...,13,...,FR,-,NaN,-,0,0,-,0,0,-
447,15,EP2144221B1,Biodegradable film for advertisement,权利终止,2009-07-07,"LEE, IN KEE | KIM, SEONG JIN",EP2009008878,"1. A biodegradable film for advertisement, co...",AT863302T | EP2144221A1 | EP2144221B1 | JP2010...,6,...,KR,CN103282589A | CN103282589B | CN105173383A | C...,NaN,EP1116748A1 | JP2004188884A | JP2007262189A | ...,4,1,-,5,0,-
448,16,EP2144966A1,Coextrusion binders on a renewable/biodegradab...,授权,2008-05-14,"CHOPINEZ, FABRICE | LAFFARGUE, JOHANN | PRADEL...",EP2008805782,19_x000D_\nREVENDICATIONS_x000D_\n1. Compositi...,AT592757T | CN101679731A | CN101679731B | DE60...,13,...,FR | US,-,NaN,-,0,0,-,0,0,-


In [2]:
data = df

In [3]:
# 构建边列表
edges = []
for row in data.iterrows():
    _, patent = row
    cited_patents = patent['引用专利'].split(' | ')
    for cited_patent in cited_patents:
        if cited_patent != '-':
            edges.append((patent['公开(公告)号'], cited_patent))

from collections import Counter

# 统计第二列中每个字符串的数量
string_counts = Counter(edge[1] for edge in edges)

# 仅保留字符串数量大于等于2的行
filtered_edges = [edge for edge in edges if string_counts[edge[1]] >= 5]

# 创建有向图
G = nx.DiGraph()
G.add_edges_from(filtered_edges)

# 生成带权邻接矩阵
adj_matrix = nx.to_pandas_adjacency(G, weight='weight')

# 保存邻接矩阵到CSV文件
#adj_matrix.to_csv("专利引用网络带权邻接矩阵大于等于8.csv")

In [4]:
betweenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering_coefficient = nx.average_clustering(G)
pagerank_centrality = nx.pagerank(G)

# 将指标结果转换为DataFrame
bc_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Node', 'Betweenness Centrality'])
cc_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Node', 'Closeness Centrality'])
ec_df = pd.DataFrame(list(eigenvector_centrality.items()), columns=['Node', 'Eigenvector Centrality'])
ccf_df = pd.DataFrame([clustering_coefficient], columns=['Clustering Coefficient'])
pr_df = pd.DataFrame(list(pagerank_centrality.items()), columns=['Node', 'Pagerank Centrality'])

# 保存指标结果为CSV文件
bc_df.to_csv("betweenness_centrality.csv", index=False)
cc_df.to_csv("closeness_centrality.csv", index=False)
ec_df.to_csv("eigenvector_centrality.csv", index=False)
ccf_df.to_csv("clustering_coefficient.csv", index=False)
pr_df.to_csv("pagerank_centrality.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'betweenness_centrality.csv'

In [5]:
import pandas as pd

bc_df = pd.read_csv("betweenness_centrality.csv")
cc_df = pd.read_csv("closeness_centrality.csv")
ec_df = pd.read_csv("eigenvector_centrality.csv")
ccf_df = pd.read_csv("clustering_coefficient.csv")
pr_df = pd.read_csv("pagerank_centrality.csv")

In [6]:
average_betweenness_centrality = bc_df['Betweenness Centrality'].mean()
average_closeness_centrality = cc_df['Closeness Centrality'].mean()
average_eigenvector_centrality = ec_df['Eigenvector Centrality'].mean()
average_clustering_coefficient = ccf_df['Clustering Coefficient'].iloc[0]  # 因为 clustering_coefficient 是 DataFrame 只有一行
average_pagerank_centrality = pr_df['Pagerank Centrality'].mean()

print("Average Betweenness Centrality:", average_betweenness_centrality)
print("Average Closeness Centrality:", average_closeness_centrality)
print("Average Eigenvector Centrality:", average_eigenvector_centrality)
print("Average Clustering Coefficient:", average_clustering_coefficient)
print("Average Pagerank Centrality:", average_pagerank_centrality)


Average Betweenness Centrality: 7.268656036346662e-06
Average Closeness Centrality: 0.011501049524367665
Average Eigenvector Centrality: 0.011684792670353402
Average Clustering Coefficient: 0.0387214596245361
Average Pagerank Centrality: 0.0026178010471203557
